<a href="https://colab.research.google.com/github/kiruma524/CreditDerivativesSimulator/blob/need-some-editing/CreditDerivativesSimulator/templates/strategies/kalman_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
VERSION =  "v1.1" #@param {type:"string"}

In [2]:
#@title [RUN] Install required python libraries and clone the repo
# Assert for correct version

assert VERSION in ['v1.0', 'v1.1'], 'Specify correct version!' + \
                                'Available versions are "v1.0" and "v1.1"'

!pip install wandb -q
!git clone -b $VERSION https://github.com/vladargunov/CreditDerivativesSimulator.git

# Import necessary classses
%cd CreditDerivativesSimulator
from src.base_strategy import BaseStrategy
from src.simulator import Simulator

fatal: destination path 'CreditDerivativesSimulator' already exists and is not an empty directory.
/content/CreditDerivativesSimulator


In [3]:
!pip install pykalman
from pykalman import *
import pandas as pd
import numpy as np

class SimpleKalmanFilter(BaseStrategy):
  def train_model(self, train_data):
    self.train_data = train_data[['er_itraxx_main_long', 'er_itraxx_main_short']]
    self.train_data = self.train_data.dropna()
    self.kf = KalmanFilter(transition_matrices = [[1, 1], [0, 1]], observation_matrices = [[0.1, 0.5], [-0.3, 0.0]])
    measurements = self.train_data.to_numpy()
    self.kf = self.kf.em(measurements, n_iter = 5)
    (self.filtered_state_means, self.filtered_state_covariances) = self.kf.filter(measurements)
    self.covariance = self.filtered_state_covariances[-1][0][1]

  def trade(self, daily_data) -> dict:
    daily_data = pd.DataFrame([daily_data])
    daily_data = daily_data[['er_itraxx_main_long', 'er_itraxx_main_short']]
    self.train_data = pd.concat([self.train_data, daily_data])
    new_measurements = daily_data.to_numpy()
    new_means, new_covariances = self.kf.filter_update(self.filtered_state_means[-1], self.filtered_state_covariances[-1], new_measurements[-1])
    #print('11111111',self.filtered_state_covariances)
    self.filtered_state_means = np.append(self.filtered_state_means, new_means)
    self.filtered_state_covariances = np.append(self.filtered_state_covariances, new_covariances)
    #print('2222222222', self.filtered_state_covariances)
    self.covariance = self.filtered_state_covariances[-1]
    if self.covariance > 0:
      return {'er_itraxx_main_short': -0.3, 'er_itraxx_main_long': 0.7}
    else:
      return {'er_itraxx_main_short': 0.3, 'er_itraxx_main_long': -0.7}


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Create an instance of your strategy and of simulator
my_strategy = SimpleKalmanFilter()


# If you use wandb, you will need to past an API key from your wandb account
sim = Simulator(use_wandb=True, 
                debug_mode=False, 
                train_test_split_time='2019-01-02', # submit date in format 'yyyy-mm-dd'
                run_name='Kalman_filters_with_no_update',
                project_name='test') # or 'Final'

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aqua_524 (cmf-credit-derivatives). Use `wandb login --relogin` to force relogin


In [5]:
# Available tickers in new version
sim.get_availiable_assets()

['er_itraxx_xover_long',
 'sx5e',
 'ieac',
 'er_itraxx_xover_short',
 'ihyg',
 'vix',
 'spx',
 'er_itraxx_main_long',
 'er_cdx_ig_short',
 'itraxx_xover_generic',
 'cdx_ig_generic',
 'er_cdx_ig_long',
 'lqd',
 'itraxx_main_generic',
 'er_itraxx_main_short',
 'hyg']

In [6]:
# Simulate your strategy
sim.simulate(strategy=my_strategy)

100%|██████████| 941/941 [00:04<00:00, 195.53it/s]



Final value of portfolio 1.0257151818698989
Sharpe of the porfolio 0.0034214759365425256
Process completed!


annualised_return (252 days),██▆▆▄▅▄▄▄▄▄▄▃▂▂▃▂▃▃▂▂▂▂▄▃▃▃▃▃▃▃▃▃▂▂▂▂▁▂▁
annualised_return (total),██▆▅▃▄▄▃▃▃▃▃▃▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
daily_return,▁▄▄▃▃▂▁▂▃▃▂▃▂█▁▅▂▃▃▂▃▃▃▃▂▂▃▂▃▂▃▂▂▂▂▂▃▃▄▂
drawdown (252 days),█▇▆▅▅▅▄▄▄▃▃▄▅▂▂▂▂▂▁▁▁▁▁▁▄▅▆▆▆▆▇▇▇▇▆▆▆▄▄▄
drawdown (total),█▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
portfolio_value,▁▂▃▄▃▄▄▅▄▅▅▆▆▃▄▅▅▆▆▆▆▆▆▆▇▇▇██████▇▇▆▆▄▆▅
sharpe,▁
annualised_return (252 days),-9e-05
annualised_return (total),3e-05
daily_return,-0.00157
drawdown (252 days),-0.03023


# Debugging tools

Now you can access the training and testing data without running a strategy. Firstly initialise a simulator, and then call these functions (without calling 'sim.simulate'):

In [7]:
# Initialise a simulator for getting train and test data
debug_sim = Simulator(use_wandb=False,
                      train_test_split_time='2019-01-02')

In [8]:
# Get training data
training_data = debug_sim.get_training_data()

training_data.head()

,er_itraxx_xover_long,sx5e,ieac,er_itraxx_xover_short,ihyg,vix,spx,er_itraxx_main_long,er_cdx_ig_short,itraxx_xover_generic,cdx_ig_generic,er_cdx_ig_long,lqd,itraxx_main_generic,er_itraxx_main_short,hyg
Date,,,,,,,,,,,,,,,,
2007-03-20,100.0000,4079.5200,NaN,100.0000,NaN,13.27,1410.94,100.0000,100.0000,NaN,NaN,100.0000,58.9123,NaN,100.0000,NaN
2007-03-21,100.2951,4078.2900,NaN,99.7046,NaN,12.19,1435.04,100.0287,99.9150,NaN,NaN,100.0850,59.0875,NaN,99.9713,NaN
2007-03-22,100.8037,4170.0801,NaN,99.1985,NaN,12.93,1434.54,100.0833,99.8854,NaN,NaN,100.1147,58.9232,NaN,99.9168,NaN
2007-03-23,100.8924,4191.5801,NaN,99.1111,NaN,12.95,1436.11,100.1090,99.8892,NaN,NaN,100.1108,58.7699,NaN,99.8911,NaN
2007-03-26,100.7493,4142.1099,NaN,99.2515,NaN,13.16,1437.50,100.1063,99.9317,NaN,NaN,100.0683,58.8794,NaN,99.8937,NaN


In [9]:
# Get test data at each trading day:

# Initialise an iterator
test_data = debug_sim.get_test_data()

In [10]:
# Get what trade() function receives at each step
# Each time you call this cell, you will receive data for next
# trade day, to reset test data call previous cell
for trade_day in test_data:
  print(trade_day)
  break

{'er_itraxx_xover_long': 191.6516, 'sx5e': 2993.1799, 'ieac': 123.4972, 'er_itraxx_xover_short': 43.5902, 'ihyg': 86.4939, 'vix': 23.22, 'spx': 2510.03, 'er_itraxx_main_long': 110.8989, 'er_cdx_ig_short': 88.1972, 'itraxx_xover_generic': 361.295, 'cdx_ig_generic': 88.788, 'er_cdx_ig_long': 108.1261, 'lqd': 101.7235, 'itraxx_main_generic': 89.224, 'er_itraxx_main_short': 85.8357, 'hyg': 67.6908}


# Details about the computation of Sharpe, Maximum Drawdown, and Annualised return

- Sharpe is calculated by the following formula:

$$ Sharpe = \frac{average \; daily \; return - risk \; free \;rate}{standard \; deviation \; daily \; return} $$

where risk free rate is computed as an average of 1 month treasury yields over the trading interval, which is then converted to an average daily rate by the formula

$$ risk \; free \;rate \; daily = (1 + risk \; free \;rate \; annual)^{1 / 365} - 1 $$

- Maximum Drawdown is computed by the formula:

$$ Maximum \; Drawdown = \frac{lowest \; value - highest \; value}{highest \; value} $$

where the highest and lowest values are specified over the trailing period.

- Annualised return is computed as:

$$ Annualised \; return = [(1 + daily \; return_1) * (1 + daily \; return_2) * ... * (1 + daily \; return_n) ]^{1 / n} - 1 $$

Where n is chosen according to the prespecified period.
